In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = [12, 6]

In [3]:
data_path = "../../data/origineel"

if not Path(data_path).is_dir():
    raise Warning("Data path does not exist")

In [5]:
data_file_paths = [p for p in Path(data_path).iterdir() if p.suffix==".csv"]

In [6]:
cableconfig = pd.read_csv(data_file_paths[3], sep=";")
partialdischarges = pd.read_csv(data_file_paths[4], sep=";")
warning = pd.read_csv(data_file_paths[5], sep=";")

In [7]:
# Namen van de kolommen
time_col, location_col, charge_col = partialdischarges.columns

# Check that each row either has location and charge undefinied, or both defined:
for i in np.arange(len(partialdischarges))[np.isnan(partialdischarges[location_col]) != np.isnan(partialdischarges[charge_col])]:
    raise Warning(f"Row {i} has missing values")

# Lijst met bools die aangeeft of er op dat tijdstip een PD was
pd_occured = ~np.isnan(partialdischarges[location_col])

# De drie kolommen; alleen op tijdstippen met PD
locations = partialdischarges[location_col][pd_occured]
charges = partialdischarges[charge_col][pd_occured]
times = partialdischarges[time_col][pd_occured].apply(datetime.datetime.fromisoformat)

# Het algoritme

Het algoritme is gebaseerd op DBSCAN (https://en.wikipedia.org/wiki/DBSCAN#Algorithm)

In [8]:
def distance(x,y):
    return abs(x-y)

def neighbors(DB, distfunc, Q, eps):
    neighborhood=[]
    for P in DB: 
        if distance(Q,P)<eps:
            neighborhood.append(P)
    return neighborhood

def DBSCAN(DB, distfunc, eps, minPts):
    cluster = []
    noise = []
    undefined = []
    for p in DB:
        undefined.append(p)
    for p in DB:
        if p not in undefined:
            continue
        N = neighbors(DB, distfunc, p, eps)
        if len(N) < minPts:
            noise.append(p)
            undefined.remove(p)
            continue
        cluster.append(p)
        undefined.remove(p)
        S = N
        S.remove(p)
        while len(list(set(S).intersection(undefined)))>0:
            for q in S:
                if q in noise:
                    noise.remove(q)
                    cluster.append(q)
                if q not in undefined:
                    continue
                cluster.append(q)
                undefined.remove(q)
                N = neighbors(DB, distfunc, q, eps)
                if len(N) > minPts:
                    S = list(set(S).union(N))
        print(sum(cluster)/len(cluster))
        cluster = []
  
    
                
                
        
    

In [13]:
DBSCAN(locations, distance, 3, 375)

349.0109393694159
605.926646845053


Dit duurde 8 minuten. De parameters zijn willekeurig gekozen, dus het resultaat zal waarschijnlijk niet kloppen. 